<a href="https://colab.research.google.com/github/Masterjun12/Pytorch-tutorials-/blob/main/pytorch_tutorial_quick_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size) # 배치 사이즈로 쪼개진 개수임 전체 X
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [15]:
total_data_count = len(train_dataloader) * batch_size
print("Total number of data samples:", total_data_count)


Total number of data samples: 60032


In [11]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [16]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # 모델을 평가모드로
    test_loss, correct = 0, 0 # 손실 정확도 초기화
    with torch.no_grad():  # torch.no_grad() 블록 안에서 연산
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301426  [   64/60000]
loss: 2.286018  [ 6464/60000]
loss: 2.271058  [12864/60000]
loss: 2.261555  [19264/60000]
loss: 2.248059  [25664/60000]
loss: 2.223335  [32064/60000]
loss: 2.225468  [38464/60000]
loss: 2.189757  [44864/60000]
loss: 2.196785  [51264/60000]
loss: 2.167226  [57664/60000]
Test Error: 
 Accuracy: 46.9%, Avg loss: 2.152049 

Epoch 2
-------------------------------
loss: 2.171222  [   64/60000]
loss: 2.149848  [ 6464/60000]
loss: 2.100179  [12864/60000]
loss: 2.107801  [19264/60000]
loss: 2.052013  [25664/60000]
loss: 2.010288  [32064/60000]
loss: 2.024553  [38464/60000]
loss: 1.944860  [44864/60000]
loss: 1.967987  [51264/60000]
loss: 1.889442  [57664/60000]
Test Error: 
 Accuracy: 51.2%, Avg loss: 1.877670 

Epoch 3
-------------------------------
loss: 1.922189  [   64/60000]
loss: 1.880049  [ 6464/60000]
loss: 1.773573  [12864/60000]
loss: 1.807111  [19264/60000]
loss: 1.694875  [25664/60000]
loss: 1.661247  [32064/600

In [19]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth") # 모델 저장

Saved PyTorch Model State to model.pth


In [20]:
model = NeuralNetwork().to(device) # 모델 불러오기
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [21]:
classes = [ # 레이블들
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [22]:
model.eval()
x, y = test_data[0][0], test_data[0][1] # 데이터와 레이블
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    # argmax 확률 값 가장 높은 인덱스
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
